# TransparenceMix

In [ ]:
#| default_exp transparencemix

In [ ]:
#| export
# library
from semantic_segmentation_augmentations.holemakertechnique import HoleMakerTechnique, HoleMakerRandom
from semantic_segmentation_augmentations.regionmodifier import RegionModifier
from semantic_segmentation_augmentations.iholesfilling import HolesFilling

# others
import cv2
import random
from fastai.basics import *
import numpy as np

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
class TransparenceMix(HolesFilling):
    "Defines the amount of holes, the technique used to make them and the probability of apply the technique."
    def __init__(self,
                 holes_num: int = 1, # The amount of holes to make.
                 modifier: "RegionModifier" = None, # The modifier that defines the traditional augments to apply to the selected regions.
                 hole_maker: "HoleMakerTechnique" = None, # The strategy used to make the holes.
                 p: float = 0.5): # The probability of applying this technique.
        hole_maker = hole_maker if hole_maker else HoleMakerRandom()
        super().__init__(modifier, hole_maker)
        self.holes_num = holes_num
        self.p = p

    def before_batch(self):
        "Applies the CutMix technique."
        
        if not self.training:
            return
        
        x, y = tensor(self.x).clone(), tensor(self.y).clone() # tensor is defined in fastai.basics
        for image, mask in zip(self.x, self.y):
            if random.random() < self.p:
                for _ in range(self.holes_num):
                    rand = random.randint(0, x.shape[0] - 1)
                    other_image, other_mask = x[rand], y[rand]
                    xhole, yhole = self.make_hole(mask)
                    sub_image, sub_mask = other_image[:, yhole, xhole], other_mask[yhole, xhole]
                    self.fill_hole(image, mask, xhole, yhole, [sub_image, sub_mask], transparence = True)

`TranspareceMix` allows us to fuse two images with the transparence idea. That is: if the overlapping mask has some information (it is not 0 in that position), its value is selected. Otherwise, if the overlapping mask has no information in that pixel, the information of the second image is used to fulfill the final image. Thus allows us to preserve as maximum information as possible, since these information is selected before using the background class.

In [ ]:
show_doc(TransparenceMix.before_batch)

In [ ]:
#| hide
import nbdev
nbdev.nbdev_export()